## Parte 1 - 

In [2]:
import requests

r = requests.get("https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt")
txt = r.text
wordfreq = []
ate = 0
mais = 0

for w in txt.split():
    wordfreq.append({w:txt.count(w)})
    if len(w) <= 10:
      ate +=1
    else:
      mais +=1
wordfreq.append({"MAIORES QUE 10":mais})
wordfreq.append({"ATÉ 10":ate}) 


## Grava dados

In [5]:
f=open('frequencia.txt','w')
l1=map(lambda x:str(x)+'\n', wordfreq)
f.writelines(l1)
f.close()

## Instalando o spark no meu notebook e criando uma sessão


In [9]:
%%sh
sudo pip install spark
sudo pip install pyspark

  Created wheel for spark: filename=spark-0.2.1-cp37-none-any.whl size=58740 sha256=970f6c7c2873f7eee5500fcc8461aadb9a141034c1507b92faf667693f7d80e0
  Stored in directory: /root/.cache/pip/wheels/57/a3/24/7c4c81b1cc5e82ad386965d2808543c4f5306b759da26e5629
Successfully built spark
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=68927f04ca0563c2b487d30ad811657dc78c1cf47afdf9518d2edf7b1f849054
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [10]:
import pandas as pd
import spark,pyspark
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.functions import col,when,lit,row_number
from pyspark.sql.window import Window
from datetime import date

config = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'),\
                                     ('spark.executor.cores', '3'),\
                                     ('spark.cores.max', '3'),\
                                     ('spark.driver.memory','8g')                             
                                     ] )
 
#sc = pyspark.SparkContext(conf=config)
import pandas as pd
spark = SparkSession.builder.config(conf=config).getOrCreate()

## Criei um arquivo contendo as datas das ultimas blackfridays de 2018,2019 e 2020, solução muito mais maleável, para adicionar futuras datas sem ficar inserindo valores no código. o Arquivo é blackfriday.txt

In [11]:
bf = spark.read.format("csv").option("header","true").load("blackfriday.txt")

## Leitura do csv e tratamento na tipagem dos campos data_pedido e data_nascimento_cliente_dt

In [14]:
df = spark.read.format("csv").option("header","true").load("clientes_pedidos.csv")
#Conventertendo para data
df = df.withColumn("data_pedido",f.from_unixtime('data_pedido').cast(t.DateType()))
df = df.withColumn("data_nascimento_cliente_dt",df.data_nascimento_cliente.cast(t.DateType()))
#Idade
df = df.withColumn("idade",((f.months_between(f.lit(date.today()),df.data_nascimento_cliente_dt)/12).cast(t.IntegerType())))
#Join black friday
df = df.join(bf,(df.data_pedido==bf.data),how='left')
df.show(10)

+--------------------+--------------------+-----------------------+-----------+--------------------------+-----+----------+------+
|       codigo_pedido|      codigo_cliente|data_nascimento_cliente|data_pedido|data_nascimento_cliente_dt|idade|      data|flg_bf|
+--------------------+--------------------+-----------------------+-----------+--------------------------+-----+----------+------+
|bc8b03a005d5bf742...|b07af86a4a6870737...|    1985-12-04 00:00:00| 2018-11-23|                1985-12-04|   35|2018-11-23|     1|
|19b0583adf75322cc...|eaaf6b26ef3b9712e...|    1979-11-14 00:00:00| 2018-11-23|                1979-11-14|   41|2018-11-23|     1|
|58fafb698b6d343e0...|c69f2ab5fc61484d7...|    1989-07-25 00:00:00| 2018-11-23|                1989-07-25|   31|2018-11-23|     1|
|79dd9f6c88ba32c97...|b4067845511997517...|    1953-12-14 00:00:00| 2018-11-23|                1953-12-14|   67|2018-11-23|     1|
|968806d40adf6aa8c...|7eecbc06bfec32b80...|    1985-05-03 00:00:00| 2018-11-23|    

## Cliente com mais de 2 compras na black friday. 

In [15]:
df = df.withColumn("compras_bf",f.sum(df.flg_bf).over(Window.partitionBy("codigo_cliente")))
df_black = df.where(df.compras_bf>2).toPandas()
df_black.to_csv("black_friday.csv")

In [16]:
 #df pedidos dos clientes menores que trinta
 df_3 = df.where(df.idade<30)
 df_31 = df_3.groupBy(df_3.codigo_cliente,df_3.idade).agg(
     f.count(df_3.codigo_pedido).alias("numero_pedido"),
     f.collect_list(f.struct(df_3.codigo_pedido,df_3.data_pedido)
 ).alias("lista_pedidos"))
 df_32 = df_31.toPandas()
 df_32.to_csv("menores_30.csv")

In [17]:
df_32.head(10)

,codigo_cliente,idade,numero_pedido,lista_pedidos
0,02a4cd7d6c37a0770416363e59390bad,27,2,"[(adb6a53bbe487c656628461abb5cec16, 2018-11-12..."
1,032a20d224caa31477831266a2202eba,28,1,"[(faa14a28d0a3d09c5e7a533958b7909c, 2018-11-04)]"
2,03a9b7854cd9f6d46562ef8f84dd80ca,23,2,"[(37d31c1484118912f1e048c43b496821, 2017-11-11..."
3,049b1269d968f4952fe9887acfa592f4,29,1,"[(4ea0a0b7d051bb29ed2a661eb9830fcf, 2017-11-17)]"
4,053826e38b608939a12712598f39a2a5,15,2,"[(ab0a5c3e6fe3fe612c8211adcff17011, 2018-11-10..."
5,0558f208136b714657dccede42c78608,24,1,"[(db5a6f48100653ae5c61bc773ee9fd5e, 2017-11-20)]"
6,06ac89bb75cd6978f4ef188dc63b8292,23,1,"[(5113f7f8e23479ae2145da040cb529e3, 2018-11-03)]"
7,06fb19472c334d63684bfe846ea9a052,21,1,"[(fc652e5c7e814e87761ccd7349cb846c, 2018-11-22)]"
8,09afe57a723c26e5479757b5e341205e,28,1,"[(cd1da80a9e4a2e3ec7782539e68a9928, 2018-11-16)]"
9,09b328c784f492911318f21b6eadf244,25,1,"[(b3ab7b91d29f08dcf7f83d8d9efbc4fb, 2017-11-27)]"
